# new 方法: stage1 & stage2

In [1]:
import sys
sys.path.append("..")

import time
import pandas as pd
from models import Yolov4
from tensorflow import keras
from config import yolo_config
from iou import get_iou, calculate_ciou
from utils import DataGenerator, read_annotation_lines

import numpy as np
from glob import glob
from tqdm.notebook import tqdm

import seaborn as sns
from turtle import color
from matplotlib import style
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import math
import cv2 as cv
import xml.etree.ElementTree as ET

import pickle

import os
# 使用第二張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# 載入模型

In [2]:
# yolo_config['score_threshold']=0.7
# print(yolo_config)

In [3]:
import warnings
warnings.filterwarnings('ignore')
class_name_path = '../class_names/classes.txt'
model = Yolov4(
               weight_path    =None, 
               config         =yolo_config,
               class_name_path=class_name_path, 
               )
model.load_weights(path="../model/202211/20221104_4_sopbox_0.weights")            # 換stage2模型

2023-03-07 11:42:15.903139: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 11:42:15.909848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 11:42:15.910323: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 11:42:15.910976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

nms iou: 0.5 score: 0.4


In [4]:
import pickle
testdata = "test_q"

with open('../../classification/dataset_test_everyframes/{}_stage1_everyframes_result_20230110.pk'.format(testdata), 'rb') as f:
    stage1_result = pickle.load(f)

In [5]:
stage1_result.head()

,stage1_pred_result,img_name
0,1,q_002462.png
1,1,q_002464.png
2,1,q_002465.png
3,1,q_002467.png
4,1,q_002516.png


In [6]:
row,col = stage1_result.shape
print(row,col)

1178 2


In [10]:
test_img_paths = "../../classification/dataset_test_everyframes/{}/".format(testdata)

'''
output_img: image array
detections: DataFram of box(x1,y1,x2,y2)
'''
pred_box = []
pred_box_num = 0  # 有預測框的圖像數量

start_time = time.time()
s_time     = time.time()
loop_times = []
for r in range(row):
    test_img_name = stage1_result.iloc[r:r+1]['img_name'].values[0]
    
    output_img, detections = model.predict(img_path=test_img_paths+test_img_name, 
                                           random_color=False, # 框顏色
                                            plot_img=False, # 顯示出框選圖片
                                            show_text=True, # 框上面加上類別文字
                                            box_num=1,      # 顯示框的數量
                                            return_output=True)
    if len(detections) == 1:
        pred_box_num += 1
        # 要把bbox存起來給 stage 3 用
        pred_box.extend([[test_img_name]+detections.values[0].tolist()])
    
    # 計算每張圖片計算時間
    temp_time = time.time()
    loop_times.append(temp_time-s_time)
    s_time = temp_time

    # 畫上bbox的影像
    # plt.imsave('../result_img/test_h_stage2_everyframes_result_20230110/test_{}'.format(test_img_name), output_img)
    


end_time = time.time()
print(end_time-start_time)
print('有預測框的圖像數量=', pred_box_num)

1/1 [==============================] - 0s 35ms/step
# of bboxes: 0
1/1 [==============================] - 0s 34ms/step
# of bboxes: 0
1/1 [==============================] - 0s 34ms/step
# of bboxes: 0
1/1 [==============================] - 0s 34ms/step
# of bboxes: 0
1/1 [==============================] - 0s 34ms/step
# of bboxes: 0
1/1 [==============================] - 0s 35ms/step
# of bboxes: 0
1/1 [==============================] - 0s 37ms/step
# of bboxes: 0
1/1 [==============================] - 0s 36ms/step
# of bboxes: 0
1/1 [==============================] - 0s 35ms/step
# of bboxes: 0
1/1 [==============================] - 0s 35ms/step
# of bboxes: 0
1/1 [==============================] - 0s 35ms/step
# of bboxes: 0
1/1 [==============================] - 0s 34ms/step
# of bboxes: 0
1/1 [==============================] - 0s 35ms/step
# of bboxes: 0
1/1 [==============================] - 0s 35ms/step
# of bboxes: 0
1/1 [==============================] - 0s 34ms/step
# of bboxe

In [11]:
# pd.DataFrame(pred_box,columns=['img_name','x1','y1','x2','y2','class_name','score','w','h']).to_pickle("../result_img/{}_stage2_everyframes_result_20230110/{}_stage2_everyframes_result_20230110.pickle".format(testdata,testdata))


In [20]:
print("min time on stage 2=",min(loop_times))
print("Max time on stage 2=",max(loop_times))
print("Avg time on stage 2=",sum(loop_times)/len(loop_times))

min time on stage 2= 0.0712270736694336
Max time on stage 2= 0.22298169136047363
Avg time on stage 2= 0.07474553504095417
